<a href="https://colab.research.google.com/github/56nh4/zoeywu/blob/main/Reply_Template_Messages.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#EX05-04 Reply Template Messages
- 依使用者輸入：confirm, buttons, carousel, imgCarousel，分別傳送確認模板、按鈕模板、輪播模板及圖片輪播模板。
- 模板上的按鈕能觸發不同類型的 Action Object。

In [1]:
!pip install -q pyngrok flask_ngrok
!pip install -q flask line-bot-sdk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.1/776.1 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.6/165.6 kB 12.1 MB/s eta 0:00:00


In [13]:
from google.colab import userdata
from pyngrok import ngrok
from flask_ngrok import run_with_ngrok

def ngrok_start():
  ngrok.set_auth_token(userdata.get('NGROK_AUTHTOKEN'))
  ngrok.connect(5000)
  run_with_ngrok(app)

from flask import Flask, request, abort

from linebot.v3 import WebhookHandler
from linebot.v3.exceptions import InvalidSignatureError
from linebot.v3.webhooks import MessageEvent, TextMessageContent
from linebot.v3.messaging import (
    Configuration, ApiClient, MessagingApi,
    ReplyMessageRequest,
    TextMessage,
    TemplateMessage, ConfirmTemplate, MessageAction,
    CarouselTemplate,
    CarouselColumn,
    URIAction,
    PostbackAction
)

app = Flask(__name__)

configuration = Configuration(access_token=userdata.get('LINE_CHANNEL_ACCESS_TOKEN'))
handler = WebhookHandler(userdata.get('LINE_CHANNEL_SECRET'))

@app.route("/callback", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']
    # get request body as text
    body = request.get_data(as_text=True)
    #handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)
    return 'OK'


@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
   with ApiClient(configuration) as api_client:
      line_bot_api = MessagingApi(api_client)

      action = event.message.text
      if action == 'confirm':
        reply = TemplateSendMessage(
            alt_text="這是確認視窗",
            template=ConfirmTemplate(
                text="你喜歡派大星嗎?",
                actions=[
                    MessageAction(label='是', text='我超喜歡派大星!'),
                    MessageAction(label='否', text='寶寶很喜歡，但寶寶不說')
                ]
            )
          )

      elif action == 'carousel':
          carousel_template = CarouselTemplate(
                columns=[
                  CarouselColumn(
                    thumbnail_image_url='https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRpyd0wpIQorEECPYtZgHujfnqD-i0a-HSDhQ&s',
                    title='派大星',
                    text='金剛屁屁!!',
                    actions=[
                      URIAction(label='維基百科',url='https://zh-yue.wikipedia.org/wiki/%E6%B4%BE%E5%A4%A7%E6%98%9F'),
                      MessageAction(label="投票",text="我投金剛屁屁一票!")
                    ]
                  ),
                  CarouselColumn(
                    thumbnail_image_url='https://hao9.qhimg.com/t01fff910ea5364319b.jpg',
                    title='海綿寶寶',
                    text='抓水母!!',
                    actions=[
                      URIAction(label='維基百科',url='https://zh.wikipedia.org/zh-tw/%E6%B5%B7%E7%B6%BF%E5%AF%B6%E5%AF%B6'),
                      MessageAction(label="投票",text="我投海綿一票!")
                    ]
                  )
               ]
              )




          reply = TemplateSendMessage(
              alt_text="這是輪播視窗",
              template=carousel_template
          )
      else:
        response = ask_gemini(action)
        reply = TextMessage(text=response)

      line_bot_api.reply_message(
          ReplyMessageRequest(
              reply_token=event.reply_token,
              messages=[reply]
          )
      )

ngrok_start()
if __name__ == "__main__":
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://d5b2-34-106-25-6.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040
